# Breast Ultrasound Image Classification

This notebook implements a deep learning approach for classifying breast ultrasound images using three different models: DenseNet, ResNet, and EfficientNet.

## Setup and Environment Configuration

First, we'll clone the repository and set up the environment.

In [ ]:
# Fix for directory access issues in Colab
import os
try:
    os.getcwd()  # Test if current directory is accessible
except:
    # If not, change to /content which should always exist in Colab
    print("Current directory is inaccessible, moving to /content...")
    os.chdir("/content")

# Clone the repository
!git clone https://github.com/vedant7001/Breast-cancer-Ultrasound-classification.git
%cd Breast-cancer-Ultrasound-classification

# Run the setup script - this will create all needed files
!python3 colab_setup.py

# The colab_setup.py script now creates all required utility scripts directly
# No need to explicitly run test_colab_setup.py as a separate file
print("\nSetup completed successfully!")

## Troubleshooting Google Colab Issues

If you encounter directory access errors like `getcwd: cannot access parent directories`, run this code to fix it:

```python
import os
os.chdir("/content")  # Move to /content directory which always exists
```

If you see an error that Python can't open a file because it doesn't exist, make sure you're in the right directory:

```python
# Check current directory
!pwd
# List files in current directory
!ls -la
```

If needed, you can recreate the required utility scripts by running the setup script again:

```python
%cd /content/Breast-cancer-Ultrasound-classification
!python colab_setup.py
```


## Download and Prepare Dataset

Download the Breast Ultrasound Images (BUSI) dataset from Kaggle.

In [ ]:
# Use our dataset utility script to download and set up the dataset
# This script is created by colab_setup.py
!python dataset_utils.py

# If that fails, try this manual method
# Uncomment the following lines if dataset_utils.py fails

# # Install kaggle API if needed
# !pip install -q kaggle
# 
# # Upload your kaggle.json file
# from google.colab import files
# print("Upload your kaggle.json file (from your Kaggle account settings)")
# uploaded = files.upload()
# 
# # Configure kaggle credentials
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json
# 
# # Create data directory if it doesn't exist
# !mkdir -p data/BUSI
# 
# # Download BUSI dataset
# !kaggle datasets download -d aryashah2k/breast-ultrasound-images-dataset
# !unzip -q breast-ultrasound-images-dataset.zip -d data/BUSI
# !rm breast-ultrasound-images-dataset.zip
# 
# # Check dataset structure
# !ls -la data/BUSI

## Explore Dataset

Let's explore the dataset structure and visualize some sample images.

In [ ]:
import os
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import random
import torch

# Check dataset statistics
data_dir = 'data/BUSI'
classes = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]

class_counts = {}
for class_name in classes:
    class_path = os.path.join(data_dir, class_name)
    image_files = [f for f in os.listdir(class_path) if f.endswith(('.png', '.jpg', '.jpeg'))]
    class_counts[class_name] = len(image_files)

print(f"Classes: {classes}")
print(f"Images per class: {class_counts}")

# Display sample images from each class
plt.figure(figsize=(15, 5*len(classes)))
for i, class_name in enumerate(classes):
    class_path = os.path.join(data_dir, class_name)
    image_files = [f for f in os.listdir(class_path) if f.endswith(('.png', '.jpg', '.jpeg'))]
    
    # Show 3 random images from each class
    if image_files: # Ensure there are images to sample from
      sample_images = random.sample(image_files, min(3, len(image_files)))
      for j, img_file in enumerate(sample_images):
          img_path = os.path.join(class_path, img_file)
          img = Image.open(img_path)
          
          plt.subplot(len(classes), 3, i*3 + j + 1)
          plt.imshow(img, cmap='gray' if img.mode == 'L' else None)
          plt.title(f"{class_name}: {img_file}")
          plt.axis('off')
                
plt.tight_layout()
plt.show()

## Model Training

Train the three model architectures on the BUSI dataset.

In [ ]:
# Check GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    !nvidia-smi

In [ ]:
# Train DenseNet model
!python main.py --config config.json --model densenet --epochs 20

In [ ]:
# Train ResNet model
!python main.py --config config.json --model resnet --epochs 20

In [ ]:
# Train EfficientNet model
!python main.py --config config.json --model efficientnet --epochs 20

## Model Evaluation and Comparison

Evaluate all three models and compare their performance.

In [ ]:
# Evaluate all models using the main script with evaluation flags
!python main.py --config config.json --evaluate --models densenet resnet efficientnet

## Visualization with Grad-CAM

Visualize model predictions using Grad-CAM for interpretability.

In [ ]:
# Visualize model predictions with Grad-CAM using the main script
# Example for DenseNet, change --model to resnet or efficientnet for others
!python main.py --config config.json --visualize --model densenet

## Download Results

Download the experimental results and trained models.

In [ ]:
from google.colab import files

# Zip results and models
!zip -r results_and_models.zip experiments/ checkpoints/

# Download zip file
files.download('results_and_models.zip')